# What is content based filtering

It will give personalised recommendations by calculating similarities between the songs based on meta data viz  tags, artist, attributes(loudness, danceability, tempo, keys etc..)

for eg we give a "song A" to the recommender system. 
We will Vectorizing all songs in DB and using Cosine similarity which will give an output between -1 to 1. 1 being most similar vectors, 0 being dissimilar, -1 being complete opposite. 

 

# Data Cleaning
Drop Genre as more than 50% data is MAR 
Null tag rows to be replaced with "No tags"
data to be converted to lower case

Now when we calculate Similarity , Non unique columns like  song names, ID columns will be dropped, as we only want similar attributes to vectorize the data so that we can recommend based on top Similairty score. 

In [1]:
import kagglehub

path = kagglehub.dataset_download("undefinenull/million-song-dataset-spotify-lastfm")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\ljm47\.cache\kagglehub\datasets\undefinenull\million-song-dataset-spotify-lastfm\versions\1


In [3]:
import os
from pathlib import Path
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
data_path = Path("C:/Users/ljm47/.cache/kagglehub/datasets/undefinenull/million-song-dataset-spotify-lastfm/versions/1")

song_data_path = data_path/ 'Music info.csv'
user_data_path = data_path/ 'User Listening History.csv'

#load songs data
df_songs = pd.read_csv(song_data_path)
df_songs.head(3)

,track_id,name,artist,spotify_preview_url,spotify_id,tags,genre,year,duration_ms,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,TRIOREW128F424EAF0,Mr. Brightside,The Killers,https://p.scdn.co/mp3-preview/4d26180e6961fd46...,09ZQ5TmUG8TSL56n0knqrj,"rock, alternative, indie, alternative_rock, in...",NaN,2004,222200,0.355,...,1,-4.360,1,0.0746,0.001190,0.000000,0.0971,0.240,148.114,4
1,TRRIVDJ128F429B0E8,Wonderwall,Oasis,https://p.scdn.co/mp3-preview/d012e536916c927b...,06UfBBDISthj1ZJAtX4xjj,"rock, alternative, indie, pop, alternative_roc...",NaN,2006,258613,0.409,...,2,-4.373,1,0.0336,0.000807,0.000000,0.2070,0.651,174.426,4
2,TROUVHL128F426C441,Come as You Are,Nirvana,https://p.scdn.co/mp3-preview/a1c11bb1cb231031...,0keNu0t0tqsWtExGM3nT1D,"rock, alternative, alternative_rock, 90s, grunge",RnB,1991,218920,0.508,...,4,-5.783,0,0.0400,0.000175,0.000459,0.0878,0.543,120.012,4


In [5]:
df_songs.shape

(50683, 21)

In [5]:
df_songs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50683 entries, 0 to 50682
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   track_id             50683 non-null  object 
 1   name                 50683 non-null  object 
 2   artist               50683 non-null  object 
 3   spotify_preview_url  50683 non-null  object 
 4   spotify_id           50683 non-null  object 
 5   tags                 49556 non-null  object 
 6   genre                22348 non-null  object 
 7   year                 50683 non-null  int64  
 8   duration_ms          50683 non-null  int64  
 9   danceability         50683 non-null  float64
 10  energy               50683 non-null  float64
 11  key                  50683 non-null  int64  
 12  loudness             50683 non-null  float64
 13  mode                 50683 non-null  int64  
 14  speechiness          50683 non-null  float64
 15  acousticness         50683 non-null 

In [6]:
# Perce3ntage of missing values
x = df_songs.isna().sum()
y= x[5]/df_songs.shape[0]
z= x[6]/df_songs.shape[0]
print(f'Percentage of tags data missing in dataset is = ', y*100)
print(f'Percentage of genre data missing in dataset is = ', z *100)

Percentage of tags data missing in dataset is =  2.223625278693053
Percentage of genre data missing in dataset is =  55.906319673263226


In [7]:
(
    df_songs.duplicated(subset="spotify_id").sum()
)

9

In [8]:
df_songs.drop_duplicates(subset=['spotify_id','year', 'duration_ms'], inplace=True)
print(f' Dataset now shows less 9 duplicate rows' , df_songs.shape)

 Dataset now shows less 9 duplicate rows (50674, 21)


In [9]:
df_songs.reset_index(drop=True, inplace=True)
# reseeting index so that no issues are caused during filtering later on 

In [10]:
# Now removing columns for Filtering which are unqiue as  there wont be any (cosine) Similarity 
# DATAFRAME FOR COLLABORATIVE FILTERING.

cols_to_remove = ['track_id', 'name', 'spotify_preview_url','spotify_id', 'genre']

df_colab_filtering = df_songs.drop(columns=cols_to_remove)
df_colab_filtering.head(2)

,artist,tags,year,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,The Killers,"rock, alternative, indie, alternative_rock, in...",2004,222200,0.355,0.918,1,-4.360,1,0.0746,0.001190,0.0,0.0971,0.240,148.114,4
1,Oasis,"rock, alternative, indie, pop, alternative_roc...",2006,258613,0.409,0.892,2,-4.373,1,0.0336,0.000807,0.0,0.2070,0.651,174.426,4


In [11]:
df_colab_filtering.isna().sum()

artist                 0
tags                1126
year                   0
duration_ms            0
danceability           0
energy                 0
key                    0
loudness               0
mode                   0
speechiness            0
acousticness           0
instrumentalness       0
liveness               0
valence                0
tempo                  0
time_signature         0
dtype: int64

In [12]:
df_colab_filtering['tags'].fillna('no_tags', inplace=True) # Fill all NAN values in tags with No tags.
df_colab_filtering.isna().sum()

artist              0
tags                0
year                0
duration_ms         0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
time_signature      0
dtype: int64

In [13]:
popular_tags =  df_songs.loc[:,'tags'].str.lower().str.split(',').explode().str.strip().value_counts()
most_popular_tags = popular_tags[popular_tags > 1000].index
most_popular_tags

# storing the most popular tags where tags appeared more than 1000 times in dataset


Index(['rock', 'indie', 'electronic', 'alternative', 'pop', 'female_vocalists',
       'alternative_rock', 'indie_rock', 'metal', 'classic_rock',
       'singer_songwriter', 'experimental', 'chillout', 'ambient', 'folk',
       'punk', 'british', 'instrumental', 'hard_rock', '80s', 'dance', '90s',
       'acoustic', 'hip_hop', 'death_metal', 'jazz', 'mellow', 'soul',
       'punk_rock', 'heavy_metal', 'beautiful', 'indie_pop', 'hardcore', '70s',
       '00s', 'metalcore', 'love', 'downtempo', 'chill', 'trip_hop',
       'progressive_rock', 'piano', 'psychedelic', 'emo', 'blues', 'electro',
       '60s', 'post_punk', 'industrial', 'new_wave', 'britpop',
       'progressive_metal', 'oldies', 'american', 'funk', 'post_hardcore',
       'male_vocalists', 'soundtrack', 'country', 'melodic_death_metal',
       'grunge', 'blues_rock', 'guitar', 'lounge', 'rap', 'swedish', 'rnb',
       'synthpop', 'thrash_metal', 'reggae', 'pop_rock', 'post_rock',
       'black_metal', 'techno', 'gothic', 'id

In [14]:
! pip install category_encoders

In [15]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from category_encoders.count import CountEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer

#A Count Encoder transforms categorical values into counts of how often they appear in a column. Useful in Year columns, as it will be a high cardinality column if we use OHE
#This is useful for converting categorical variables to numeric form for machine learning models.
#Reduces dimensionality as comapred to OHE 
#similar to map function : df['tags_count'] = df['tags'].map(df['tags'].value_counts()) rock(0) = 95, metal (1) - 80 eg. 

In [16]:
df_colab_filtering.head(3)

,artist,tags,year,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,The Killers,"rock, alternative, indie, alternative_rock, in...",2004,222200,0.355,0.918,1,-4.360,1,0.0746,0.001190,0.000000,0.0971,0.240,148.114,4
1,Oasis,"rock, alternative, indie, pop, alternative_roc...",2006,258613,0.409,0.892,2,-4.373,1,0.0336,0.000807,0.000000,0.2070,0.651,174.426,4
2,Nirvana,"rock, alternative, alternative_rock, 90s, grunge",1991,218920,0.508,0.826,4,-5.783,0,0.0400,0.000175,0.000459,0.0878,0.543,120.012,4


In [17]:
df_colab_filtering.shape

(50674, 16)

In [18]:
# Cols to transform
frequency_encode_cols = ['year']
ohe_cols = ['artist', 'time_signature', 'key'] # catrgorical columns 
tfidf_col = 'tags' #  just pass the column name as a string
standard_scale_cols = ['duration_ms', 'loudness','tempo'] #numerical columns, scaling 
min_max_scale_cols = ["danceability","energy","speechiness","acousticness","instrumentalness","liveness","valence"] # Normalizing features for distance-based models

In [19]:
len(frequency_encode_cols + ohe_cols + standard_scale_cols + min_max_scale_cols)


14

In [20]:
df_colab_filtering.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50674 entries, 0 to 50673
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   artist            50674 non-null  object 
 1   tags              50674 non-null  object 
 2   year              50674 non-null  int64  
 3   duration_ms       50674 non-null  int64  
 4   danceability      50674 non-null  float64
 5   energy            50674 non-null  float64
 6   key               50674 non-null  int64  
 7   loudness          50674 non-null  float64
 8   mode              50674 non-null  int64  
 9   speechiness       50674 non-null  float64
 10  acousticness      50674 non-null  float64
 11  instrumentalness  50674 non-null  float64
 12  liveness          50674 non-null  float64
 13  valence           50674 non-null  float64
 14  tempo             50674 non-null  float64
 15  time_signature    50674 non-null  int64  
dtypes: float64(9), int64(5), object(2)
memor

In [21]:
transformer = ColumnTransformer(transformers= [
        ('frequence_encode', CountEncoder(normalize=True, return_df=True), frequency_encode_cols),
        ("ohe", OneHotEncoder(handle_unknown="ignore"), ohe_cols),
        ("tfidf", TfidfVectorizer(max_features=85), tfidf_col),
        ("standard_scale", StandardScaler(), standard_scale_cols),
        ("min_max_scale", MinMaxScaler(), min_max_scale_cols)
    ],remainder='passthrough',n_jobs=-1,force_int_remainder_cols=False)

In [22]:
print(transformer.get_feature_names_out)

<bound method ColumnTransformer.get_feature_names_out of ColumnTransformer(force_int_remainder_cols=False, n_jobs=-1,
                  remainder='passthrough',
                  transformers=[('frequence_encode',
                                 CountEncoder(combine_min_nan_groups=True,
                                              normalize=True),
                                 ['year']),
                                ('ohe', OneHotEncoder(handle_unknown='ignore'),
                                 ['artist', 'time_signature', 'key']),
                                ('tfidf', TfidfVectorizer(max_features=85),
                                 'tags'),
                                ('standard_scale', StandardScaler(),
                                 ['duration_ms', 'loudness', 'tempo']),
                                ('min_max_scale', MinMaxScaler(),
                                 ['danceability', 'energy', 'speechiness',
                                  'acousticness', 'in

In [23]:
transformer

,transformers,"[('frequence_encode', ...), ('ohe', ...), ...]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,-1
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,False
,verbose,0
,cols,None
,drop_invariant,False


In [24]:
transformer.fit(df_colab_filtering)

c:\Users\ljm47\anaconda3\envs\dev\lib\site-packages\sklearn\compose\_column_transformer.py:975: FutureWarning: The parameter `force_int_remainder_cols` is deprecated and will be removed in 1.9. It has no effect. Leave it to its default value to avoid this warning.
  warnings.warn(


,transformers,"[('frequence_encode', ...), ('ohe', ...), ...]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,-1
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,False
,verbose,0
,cols,None
,drop_invariant,False


In [25]:
transformed_df = transformer.transform(df_colab_filtering)
transformed_df.shape

(50674, 8431)

In [26]:
 # Now after we have transformed the dataset we need to perform COsine Similarity. The transformed df is a Sparse matrix, however cosine similairty can take sparse matrixes
transformed_df

<50674x8431 sparse matrix of type '<class 'numpy.float64'>'
	with 907911 stored elements in Compressed Sparse Row format>

In [27]:
df_songs.loc[df_songs['artist']=='Shakira'].head(3)

,track_id,name,artist,spotify_preview_url,spotify_id,tags,genre,year,duration_ms,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
1025,TRLWDVU128F932B093,"Whenever, Wherever",Shakira,https://p.scdn.co/mp3-preview/09ddeb4ae33ee6e8...,07PHBDuUmOeZ7jeKSbAbKi,"rock, pop, female_vocalists, singer_songwriter...",NaN,2012,196826,0.787,...,1,-4.967,0,0.0474,0.298,0.000005,0.206,0.860,107.674,4
2068,TRILOWN128F426080F,Underneath Your Clothes,Shakira,https://p.scdn.co/mp3-preview/6c5a56058ce04371...,07qRl4PT2lA6O3KN40McLz,"rock, pop, female_vocalists, singer_songwriter...",Pop,2013,224893,0.707,...,8,-5.293,1,0.0298,0.691,0.000000,0.103,0.407,82.784,4
2205,TRXLMFJ12903CC06F7,She Wolf,Shakira,https://p.scdn.co/mp3-preview/4dc802fd3b06fcb5...,075xFXR0JDBwFPVinG1ig5,"electronic, pop, female_vocalists, experimenta...",NaN,2009,187866,0.857,...,7,-6.480,1,0.0428,0.323,0.003220,0.314,0.868,121.994,4


In [28]:
# Input vector build

df_songs.loc[df_songs['name']== 'Whenever, Wherever']

# now filtering the search based on the Colaborative Dataset as we are ignoring columns in the df_songs dataset

song_input = df_colab_filtering[df_songs['name'] ==  'Whenever, Wherever' ]
song_input

,artist,tags,year,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
1025,Shakira,"rock, pop, female_vocalists, singer_songwriter...",2012,196826,0.787,0.828,1,-4.967,0,0.0474,0.298,0.000005,0.206,0.86,107.674,4


# As of now we have 3 main datasets
df_songs : which contains all columns

df_colab_filtering : which does not contain columns which are unique  eg : track_id, spotify_url, name of song.
When a person gives a song name, we will use the song name to fetch out the row from the Df_songs dataset . This row will then be transformed using the transformed which was fitted on the df_colab_filtering dataset to form the third dataset called transformed_df

This row which is transformed will act as the input vector and the cosine similairty will be calculated between this input vector and the third dataset which is the transformed dataset which has 8k+ columsn . 

Transformed_dataset : This dataset contains 8k+ columns as the df_colab_filtering dataset was passed throughthe Column_transformed object. it is used to to calculate Cosine_similairty.

In [29]:
input_vector = transformer.transform(song_input)
input_vector

<1x8431 sparse matrix of type '<class 'numpy.float64'>'
	with 20 stored elements in Compressed Sparse Row format>

In [30]:
input_vector.shape

(1, 8431)

In [31]:
#Calculate the similarity matrix between the above input vector variable and entire transformed dataset

from sklearn.metrics.pairwise import cosine_similarity

In [32]:
print(transformed_df.shape)
print(input_vector.shape)

(50674, 8431)
(1, 8431)


In [33]:
similarity_score = cosine_similarity(transformed_df, input_vector)

In [34]:
print(similarity_score)
print(similarity_score.shape)

print(f' Similarity score with itself', similarity_score[1025])

[[0.99999914]
 [0.99999847]
 [0.99999921]
 ...
 [0.99999877]
 [0.9999992 ]
 [0.99999891]]
(50674, 1)
 Similarity score with itself [1.]


In [35]:
top_10_song_indexes = np.argsort(similarity_score.ravel())[-11 :][::-1] #flatten the array - .ravel() , then using Slicing,finding the last 10 elements and rev ersing the elements
# np.argsort sorts the Variable's indexes in ascending order , which means it will give the indexes of the least similar song
top_10_song_indexes

array([ 1025, 12305,  6046,  6129, 17241,  6133,  7172,  6121,  6526,
       38383,  6287], dtype=int64)

In [36]:
top_10_song_names = df_songs.iloc[top_10_song_indexes]
top_10_song_names   #we got the names of most similar songs

,track_id,name,artist,spotify_preview_url,spotify_id,tags,genre,year,duration_ms,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
1025,TRLWDVU128F932B093,"Whenever, Wherever",Shakira,https://p.scdn.co/mp3-preview/09ddeb4ae33ee6e8...,07PHBDuUmOeZ7jeKSbAbKi,"rock, pop, female_vocalists, singer_songwriter...",NaN,2012,196826,0.787,...,1,-4.967,0,0.0474,0.29800,0.000005,0.2060,0.860,107.674,4
12305,TRYFVKK128F4240FE8,Why Wait,Shakira,https://p.scdn.co/mp3-preview/d78c90c5cb5626be...,0HiJFRxWme9myvUiDlqQ8q,"pop, experimental, singer_songwriter, dance",NaN,2001,221240,0.887,...,1,-5.535,0,0.0431,0.14400,0.000590,0.1230,0.399,129.943,4
6046,TRAAKDG128F42A0ECB,Hips Don't Lie,Shakira,https://p.scdn.co/mp3-preview/3859547944f57cfb...,01Yj2MCGpjZs34PRlGgz4K,"pop, female_vocalists, singer_songwriter, danc...",Pop,2001,217453,0.777,...,10,-5.867,0,0.0734,0.28400,0.000000,0.4300,0.760,100.003,4
6129,TRBAUVN128F932FEF8,Oops!...I Did It Again,Britney Spears,https://p.scdn.co/mp3-preview/7fb86827422540ad...,095uakqDYR50Uza0mxvPWB,"pop, female_vocalists, dance, 00s",Pop,2014,211786,0.751,...,1,-5.351,0,0.0435,0.34000,0.000018,0.2550,0.886,95.045,4
17241,TRMBDIR128F4279C1F,Perfect Lover,Britney Spears,https://p.scdn.co/mp3-preview/52671e54d36f077e...,1BhxPx4evrx8X02RHGrLdi,"pop, dance, rnb, 00s",Rock,2007,182680,0.718,...,1,-3.959,0,0.0360,0.35300,0.000390,0.1020,0.805,117.067,4
6133,TRDXCSH128F92ED4A1,Bootylicious,Destiny's Child,https://p.scdn.co/mp3-preview/7e327ccb1e4c52b2...,09mkdGhqb5ySKVsnkx9hy2,"pop, female_vocalists, dance, soul, hip_hop, r...",NaN,2001,207733,0.835,...,1,-4.364,0,0.2840,0.00247,0.000000,0.1710,0.586,103.358,4
7172,TRLQBEN128F92E7415,Wild Things,Alessia Cara,https://p.scdn.co/mp3-preview/c13f00088525d0b2...,0RgHkNpqtAHGGBVro6mmqD,"pop, female_vocalists",Country,2016,188493,0.741,...,1,-5.362,0,0.1080,0.01950,0.000000,0.0822,0.735,107.960,4
6121,TRGZIMZ128F930A016,La Isla Bonita,Madonna,https://p.scdn.co/mp3-preview/d8f3cafe99c1f0cd...,0rpndqrkU9y9nckNCfjcq6,"pop, female_vocalists, dance, 80s",NaN,2009,242946,0.708,...,1,-4.736,0,0.0362,0.39200,0.000001,0.0561,0.968,99.953,4
6526,TRXHWIA128E078A9BB,Cruel Summer,Bananarama,https://p.scdn.co/mp3-preview/47d13ef240a58bef...,0BmUCeyXpTrUVahHKVFxuB,"pop, female_vocalists, dance, 80s, new_wave",NaN,1984,209573,0.665,...,1,-5.828,0,0.0278,0.25000,0.008550,0.0537,0.932,108.429,4
38383,TRWUWRZ128F42ADA4A,Dreams for Plans,Shakira,https://p.scdn.co/mp3-preview/6e2c021846087a88...,2ObxMmMaDINr0ynkqW2BlY,"pop, female_vocalists, guitar, pop_rock",Pop,2005,242760,0.689,...,1,-7.427,1,0.0286,0.18000,0.000038,0.0844,0.548,96.098,4


In [46]:
song_row = top_10_song_names.loc[(top_10_song_names['name'] == 'Trick Me') & (top_10_song_names['artist'] == 'Kelis')]
song_row

# song_row = top_10_song_names.loc[top_10_song_names['name'] == 'Trick Me']
# song_row

,track_id,name,artist,spotify_preview_url,spotify_id,tags,genre,year,duration_ms,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
6287,TRZJJHG128EF34A3F9,Trick Me,Kelis,https://p.scdn.co/mp3-preview/0862413f9e28d284...,03obQpbRm2JsmEg8jnmMgL,"pop, female_vocalists, dance, soul, hip_hop, rnb",NaN,2012,206320,0.97,...,1,-4.443,0,0.157,0.0378,0.000262,0.316,0.961,107.175,4


In [39]:
def recommend(song_name, artist_name,  song_data, transformed_data, k=10):
    """
    Recommends top k songs similar to the given song based on content-based filtering.

    Parameters:
    song_name (str): The name of the song to base the recommendations on.
    songs_data (DataFrame): The DataFrame containing song information.
    transformed_data (ndarray): The transformed data matrix for similarity calculations.
    k (int, optional): The number of similar songs to recommend. Default is 10.

    Returns:
    DataFrame: A DataFrame containing the top k recommended songs with their names, artists, and Spotify preview URLs.
    """

    #filter out song data
    song_row = song_data.loc[(song_data['name'] == song_name) & song_data['artist'] == artist_name]
    if song_row.empty:
        print(" No such song in database")
    else:
        #get the song_index
        song_index = song_row.index[0]
        print(f'song index is', song_index)
        #generate the input vector
        input_vector = transformed_data[song_index]
        print(f'input vector shape', input_vector.shape)
        print(f'transformed datae shape', transformed_data.shape)
        #calculate similairty score
        similarity_score = cosine_similarity(transformed_data, input_vector)
        print(similarity_score.shape)
        top_k_song_indexes = np.argsort(similarity_score.ravel())[-k-1:][::-1]
        print(top_k_song_indexes)
        #print top k songs
        top_k_songs_names = song_data.iloc[top_k_song_indexes]
        # a= pd.DataFrame(top_k_songs_names)
        # return a.head()
        top_k_list = top_k_songs_names[['name', 'artist', 'spotify_preview_url']]       
        return top_k_list


In [40]:
recommend("Whenever, Wherever", 'Shakira', song_data=df_songs, transformed_data=transformed_df, k=10)

 No such song in database


In [7]:
import pickle
pickle.dump(df_songs, open('song_db.pkl','wb'))
